# 1. Selecting Keyphrases to Filter Text Data

**Problem**

Having seen the data, we noticed that large parts of it were not related to the Arab Spring revolutions of 2011. Spinn3r, the company providing the data, is actually promoting their scraping API by doing so.

Therefore, we must find keyphrases to filter this data based on the Tunisian and Egyptian revolutions.

**Solution Proposed**

Using **Wikipedia** articles for the Tunisian and Egyptian revolutions, we ***automatically*** extract the keyphrase that will be used for filtering. Given that the data is provided in various languages, we choose to focus on three relevant languages: **English**, **French**, and **Arabic**.

**Added Value**

In comparison with the only paper participating in the 2011 ICWSM Spinn3r Challenge, <a href='http://www.icwsm.org/2011/documents/IDC2011.pdf'>*Revolution 2.0 in Tunisia and Egypt: Reactions and Sentiments in the Online World*</a>:
1. We use **keyphrases** instead of **keywords**, by using n-grams;
2. We do not choose keyphrases manually, but extract them from Wikipedia automatically;
3. We provide insight by evaluating data not only in English, but in two additional languages: French, Tunisia's colonial-era and second most popular language, and Arabic, the native language of both the peoples of Tunisia and Egypt.

**In the following:** we first establish helper functions before compiling it in one function to get the keyphrases for each language, and each country.

## 1.1. Tokenization and Filtering based on Parts of Speech

The following are the languages that we focus on throughout the project:

In [1]:
EN = 'english'
FR = 'french'
AR = 'arabic'
LANGS = [EN, FR, AR]

We use a *Regular Expression Tokenizer* with the *NLTK* (Natural Language Toolkit) package for the textual data.

We want our keyphrases to contain **interesting words**, and therefore we eliminate stop words using the corresponding corpus from NLTK, and we eliminate verbs and adverbs using the **state-of-the-art Stanford POS Tagger**. We have to check the distinct *tag sets* of each language's POS tagger for filtering.

In [2]:
# NLTK Imports
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.tag import StanfordPOSTagger

# Setting up POS Tagger and Stop Word Lists
TOKENIZER = RegexpTokenizer(r'\w+')
STOPWORDS = dict((language, set(stopwords.words(language))) for language in LANGS)

# Setting up POS Taggers
JAR = './stanford-postagger.jar'
POS_TAGGERS = dict((language, StanfordPOSTagger(language + '.tagger', JAR)) for language in LANGS)

# Tags of Adverbs and Verbs in each language's POS Tagger
REFUSED_POS_TAGS = {EN: ['V', 'R', 'MD', 'IN', 'CD'], 
                    FR: ['ADV', 'D', 'PREF', 'V'],
                    AR: ['V']}

def tag_is_allowed(tag, language):
    """
    Determines if given tag is not a verb or adverb using REFUSED_POS_TAGS.
    :param tag: string, tag of a word as determined by POS Tagger
    :param language: string
    :return: Boolean, True if tag should be kept
    """
    for other_tag in REFUSED_POS_TAGS[language]:
        if tag.startswith(other_tag):
            return False
    return True

def preprocess_for_keyphrases(text, language):
    """
    Tokenizes, removes stopwords and digit-only tokens, removes verbs and adverbs.
    :param text: string
    :param language: string, language of text
    :return: list of strings, tokens of text
    """
    # Removing stop words, digit-only tokens
    tokens = [token for token in TOKENIZER.tokenize(text.lower()) 
              if token not in STOPWORDS[language] 
              and not token.isdigit()]
    # POS-Tagging
    tokens_with_tags = POS_TAGGERS[language].tag(tokens)
    # Handling the case of the Arabic POS Tagger, as it concatenates token and POS tag
    if language == AR:
        tokens_with_tags = [tag[1].split('/') for tag in tokens_with_tags]
    # POS-Tag filtering
    tokens = [token for token, tag in tokens_with_tags 
              if tag_is_allowed(tag, language)]
    return tokens

## 1.2. Forming N-Grams

Having formed tokens, we want now to give sense to keyphrases by forming n-grams. We use NLTK for that as well. We use the regular expression package to have the newly-formed n-grams as one token.

In [3]:
# NLTK and regexp Imports
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder
import re

def form_n_grams(tokens, threshold):
    """
    Forms tokens that are n-grams with a minimum frequency threshold.
    :param tokens: list of strings
    :param threshold: int
    :return: list of strings, n-gram'd tokens
    """
    # Bigram Finder
    finder = BigramCollocationFinder.from_words(tokens)
    finder.apply_freq_filter(threshold)
    # Bigrams
    scored_bigrams = finder.score_ngrams(BigramAssocMeasures.raw_freq)
    # Joining Bigrams in the text
    text = ' '.join(tokens)
    for (word1, word2), score in scored_bigrams:
        text = re.sub(r'\b{}\b \b{}\b'.format(word1, word2), '{}_{}'.format(word1, word2), text)
    tokens = text.split(' ')
    return tokens

We form n-grams over different rounds, each round incrementing the *n* parameter.

Given that Arabic is more agglutinative than English and French, it gets less rounds. The following are the rounds that we define to form n-grams:

In [4]:
NGRAM_RANGE = {EN: range(3), 
               FR: range(3), 
               AR: range(1,3)}

The following function forms the n-grams over rounds. To keep interesting tokens, we remove tokens that have less than 4 characters. We remove these tokens only after forming n-grams because we noticed that short unigrams can form interesting n-grams.

In [13]:
MIN_CHAR_COUNT = 4

def n_gram_rounds(tokens, language, threshold):
    """
    Forms n-grams over rounds given unigram tokens.
    :param tokens: list of strings
    :param language: string
    :param threshold: int
    :return: list of strings, n-gram tokens
    """
    for ngram_round in NGRAM_RANGE[language]:
        tokens = form_n_grams(tokens, threshold + ngram_round)
    tokens = [token for token in tokens if len(token) >= MIN_CHAR_COUNT]
    return tokens

## 1.3. Downloading Wikipedia Articles

Throughout the project, we focus on the two following countries:

In [6]:
EG = 'Egypt'
TN = 'Tunisia'

We use the following articles to extract keyphrases.

In [7]:
WIKI_ARTICLES = {TN: {EN: 'Tunisian Revolution', 
                      FR: 'Révolution tunisienne', 
                      AR: 'الثورة التونسية'},
                 EG: {EN: 'Egyptian Revolution', 
                      FR: 'Révolution égyptienne de 2011', 
                      AR: 'ثورة 25 يناير'}}

We use the *Wikipedia* Python package to extract article content.

In [8]:
# Wikipedia import
import wikipedia

def download_wikipedia_article(wiki_article, language):
    """
    Downloads the text content of a given Wikipedia article in a given language.
    :param wiki_article: string, title of article
    :param language: string
    :return: string
    """
    wikipedia.set_lang(language[:2])
    wiki = wikipedia.page(wiki_article)
    return wiki.content

## 1.4. Extracting Most Popular Tokens

Once we have formed phrases and have them as a list of strings, we use `Counter` to get the most popular phrases.

In [9]:
from collections import Counter

def get_most_popular_tokens(tokens, num_keyphrases):
    """
    Gets most popular num_keywords elements of tokens.
    :param tokens: list of strings
    :param num_keyphrases: int
    :return: list of strings
    """
    counter = Counter(tokens)
    keyphrases = [word.replace('_', ' ') for word, score in counter.most_common(num_keyphrases)]
    return keyphrases

## 1.5. Extracting Keyphrases from Wikipedia Articles

We concatenate all of the above in the following function:

In [10]:
def get_wikipedia_keyphrases(wiki_article, language, num_keyphrases=50, threshold=1):
    """
    Gets Keyphrases from Wikipedia Article.
    :param wiki_article: string, name of Wikipedia Article
    :param language: string
    :param num_keyphrases: int, number of keyphrases to extract
    :param threshold: int, n-gram formation threshold
    :return: list of strings, keyphrases
    """
    wiki_content = download_wikipedia_article(wiki_article, language)
    unigram_tokens = preprocess_for_keyphrases(wiki_content, language)
    ngram_tokens = n_gram_rounds(unigram_tokens, language, threshold)
    keyphrases = get_most_popular_tokens(ngram_tokens, num_keyphrases)
    return keyphrases

We extract 50 keyphrases, per country, and per language:

In [14]:
keyphrase_dict = dict((country, 
                       dict((language, get_wikipedia_keyphrases(WIKI_ARTICLES[country][language], language)) 
                            for language in WIKI_ARTICLES[country])) 
                      for country in WIKI_ARTICLES)

## 1.6. Examples of Keyphrases

In the following, we provide an example of keyphrases for English in **Tunisia**. The most popular is `ben ali`, the last name of Tunisia's ousted president. The phrase `sidi bouzid` designates the city where protests started in January 2011.

In [15]:
keyphrase_dict[TN][EN][:10]

['ben ali',
 'tunisia',
 'arab world',
 'protests',
 'prime minister',
 'sidi bouzid',
 'tunisian',
 'security forces',
 'foreign minister',
 'january']

Hereafter is an sample of keyphrases for English in **Egypt**. The most popular one is `tahrir square`, a public square in the Egyptian capital, Cairo, where the protesters camped throughout the protests. The `muslim brotherhood` is the opposition party that later came into power. Here, the Egyptian ousted president's full name, `hosni mubarak`, comes up as one of the keyphrases.

In [16]:
keyphrase_dict[EG][EN][:10]

['tahrir square',
 'muslim brotherhood',
 'revolution',
 'egyptian revolution',
 'hosni mubarak',
 'social media',
 'protesters',
 'prime minister',
 'protests',
 'human rights']

## 1.7. Comparing Texts to Keyphrases

Given these keyphrases, we want to estimate how many of those appear in a given text:

In [35]:
def keyphrase_score(country, language, text):
    """
    Computes how many keyphrases for the country and language are in the text.
    :param country: string
    :param language: string
    :param text: string
    :return: int
    """
    score = 0
    for keyphrase in keyphrase_dict[country][language]:
        if keyphrase in filtered_text:
            score += 1
    return score

We use this function to compute the keyphrase scores per country and for a given language and a given text:

In [36]:
def compute_keyphrase_scores(text, language):
    """
    Computes keyphrase scores for all countries given text and language.
    Returns False if all scores are zero.
    :param text: string
    :param language: string
    :return: dict or boolean
    """
    filtered_text = ' '.join(preprocess_for_keyphrases(text, language))
    data = {}
    score_sum = 0
    for country in keyphrase_dict:
        score = keyphrase_score(country, language, filtered_text)
        data[country] = score
        score_sum += score
    if score_sum == 0:
        return False
    else:
        return data

# 2. Sampling and Filtering the 2.1 TB of Compressed Data

**Steps to Reach Text Data**

The files are compressed in `.tar.gz` format. The steps to reach text data are more complex than we thought:
1. We downloaded a sample, and the decompressed files are in `.protostream` format. This format **lacks documentation**. A `.protostream` file is a stream of Google's `protobuf` format.
2. The `.protostream` format requires `.proto` files to extract them. We found these files on the internet, not on Spinn3r's website. This format was originally made for Java, and has difficult compatibility with Python.
3. We installed the C++ `protoc` library to make `.py` files out of `.proto` files. These `.py` files contain Python classes that we can use to read the data. The `.proto` and `.py` files can be found on our GitHub repo.
4. We found the code to read the text data on a *Stackoverflow* answer rated `-1`, rather than on Spinn3r's website or the `protobuf` documentation.

**Sample Evaluated**

We downloaded compressed samples of about **40 GB** for the days of **January 14, 2011** and **February 11, 2011**, respectively the days of the resignation of the presidents of Tunisia and Egypt.

**In the following:** we first read the data and filter it based on the keywords found in the first section.

## 2.1. Reading a Single `.protostream` File

We first import the `.py` files that we got out of the `.proto` files. The `spinn3rApi_pb2.py` file contains the classes needed to parse the bytes of a given internet entry. The `protoStream_pb2.py` file contains the classes needed to separate entries inside a `.protostream` file, most notably the `ProtoStreamDelimiter` class.

In [17]:
import spinn3rApi_pb2 as proto_api
import protoStream_pb2 as proto_stream

DELIMITER = proto_stream.ProtoStreamDelimiter()

We will need the following decoder to separate entries in a `.protostream` file.

In [18]:
from google.protobuf.internal.decoder import _DecodeVarint32

# length, pos = decoder(file, pos) => pos is starting point, length is the length of the record in file
DECODER = _DecodeVarint32

In the files we find, we find the languages as two-letter language codes. We will need a mapping to full-length name:

In [37]:
CODE_TO_LANG = dict((language[:2], language) for language in LANGS)

We use the following functions to read a single `.protostream` file onto a list of `dict` instances. We use `BeautifulSoup` for some `html` data, that is itself `zlib`-encoded.

In [38]:
import zlib
from bs4 import BeautifulSoup

def try_to_get_content(entry):
    """
    Attempts to get content from an entry.
    :param entry: contentApi.Entry
    :return: string
    """
    try:
        return BeautifulSoup(zlib.decompress(entry.feed_entry.content.data), "html.parser").text
    except:
        return ""

def read_protostream_file(file):
    """
    Reads a single .protostream file.
    :param file: bytes
    :return: list of dictionaries
    """
    data_rows = []
    
    length, pos = decoder(file, 0)
    pos += length

    while pos < len(file):
        length, pos = decoder(file, pos)
        DELIMITER.ParseFromString(file[pos:pos + length])
        if DELIMITER.delimiter_type == DELIMITER.END:
            break
            
        # This means we found an Entry
        elif DELIMITER.delimiter_type == DELIMITER.ENTRY:
            pos += length
            length, pos = decoder(file, pos)
            
            # The Entry Class enables to read the contents of the record
            entry = proto_api.Entry()
            entry.ParseFromString(file[pos:pos + length])
            
            lang_code = entry.feed_entry.lang[0].code
            
            # Proceeding if the language is English, French or Arabic
            if lang_code in CODE_TO_LANG:
                post_title = entry.feed_entry.title
                post_content = try_to_get_content(entry)
                data = compute_keyphrase_scores(post_title + ' ' + post_content, 
                                                CODE_TO_LANG[lang_code])
                if data:
                    # Language Code
                    data['Lang_Code'] = lang_code
                    
                    # Post Title
                    data['Post_Title'] = post_title
                    
                    # Post Content
                    if post_content:
                        data['Post_Content'] = post_content
                    
                    # Language Probability
                    try:
                        data['Lang_Prob'] = entry.feed_entry.lang[0].probability
                    except:
                        pass

                    # Link
                    try:
                        data['Post_Link'] = entry.feed_entry.link[0].href
                    except:
                        pass

                    # Author Name
                    try:
                        data['Author_Name'] = entry.feed_entry.author[0].name
                    except:
                        pass

                    # Author Link
                    try:
                        data['Author_Link'] = entry.feed_entry.author[0].link[0].href
                    except:
                        pass

                    # Date & Time
                    try:
                        data['Datetime'] = entry.feed_entry.last_published
                    except:
                        pass

                    # Identifier
                    try:
                        data['Identifier'] = entry.feed_entry.identifier
                    except:
                        pass

                    # Spam Probability
                    try:
                        data['Spam'] = entry.feed_entry.spam_probability
                    except:
                        pass

                    # Publisher Type
                    try:
                        data['Type'] = entry.source.publisher_type
                    except:
                        pass

                    # Category
                    try:
                        data['Category'] = entry.feed_entry.category
                    except:
                        pass

                    data_rows.append(data)
        pos += length
    return data_rows

## 2.2. Reading the Compressed `.tar.gz` Files

We downloaded the files in an external harddrive. We set up the following variables:

In [39]:
PREFIX = '/Volumes/Khalil Mrini/'
FILES = '01-14-OTHER 01-14-SOCIAL_MEDIA 02-11-OTHER 02-11-SOCIAL_MEDIA'.split(' ')
EXTENSION = '.tar.gz'

To read a `.tar.gz` file without decompressing it, we use the following package:

In [40]:
import tarfile

In the following we read the `.tar.gz` files:

In [ ]:
data_rows = []

for file_name in FILES:
    print('Opening', file_name, '...')
    tar = tarfile.open(PREFIX + file_name + EXTENSION, "r:gz")
    members = tar.getmembers()
    member_count = len(members)
    print('There are {} files in {}.'.format(member_count, file_name + EXTENSION))
    for member_index in range(member_count):
        print(member_index, 'out of', member_count, end='\r', flush=True)
        file_data = tar.extractfile(members[member_index])
        if file_data is not None:
            content = file_data.read()
            data_rows.extend(read_protostream_file(content))

Opening 01-14-OTHER ...


## 2.3. Putting the Collected Data in a `DataFrame`

After having collected and filtered the data, we put it in a `Pandas` `DataFrame`.

In [ ]:
import pandas as pd

df = pd.DataFrame(data_rows)